In [1]:
import affine as af

In [2]:
import random
all_miners = await af.miners(no_null=True)
selected_uids = random.sample(list(all_miners.keys()), 2)
print (selected_uids)

[5, 6]


In [3]:
miner1 = all_miners[selected_uids[0]]
miner2 = all_miners[selected_uids[1]]
print(miner1, miner2)

uid=5 hotkey='5Hjnirznzz3rYKodKCR2F2qzXC8ZjwZbUrt8w7CdbcPgYXMV' model='unsloth/gemma-3-4b-it' block=5807815 chute={'chute_id': '4d636d79-623c-5688-b57c-264f9e53d85d', 'name': 'unsloth/gemma-3-4b-it', 'public': True, 'version': '8ff77f39-79f4-5aac-b2bd-03e37b7a3556', 'tool_description': None, 'slug': 'chutes-unsloth-gemma-3-4b-it', 'standard_template': 'vllm', 'cord_ref_id': None, 'created_at': '2025-03-12T20:06:51.488258Z', 'updated_at': '2025-05-16T13:15:12.915543Z', 'image': {'image_id': '1075450f-1c30-522e-a58a-3a7c775a008c', 'name': 'sglang', 'tag': '0.4.6.post4', 'public': True, 'status': 'built and pushed', 'created_at': '2025-05-14T16:04:26.948160Z', 'build_started_at': '2025-05-14T16:04:28.960327Z', 'build_completed_at': '2025-05-14T16:24:14.068098Z', 'user': {'username': 'chutes', 'user_id': 'dff3e6bb-3a6b-5a2b-9c48-da3abcd5ca5f', 'logo_id': '24ef9084-7739-400c-aab6-135463a8c2d9', 'created_at': '2024-12-03T11:31:39.181725Z', 'logo': 'https://logos.chutes.ai/logos/24ef9084-7739

In [5]:
results = await af.run(
    challenges = await af.envs.sat.SAT().many(1),
    miners = [miner1, miner2]
)
print (results)

AF |████████████████████████████████████████| 2/2 [100%] in 2:10.6 (0.02/s) 
[Result(miner=Miner(uid=5, hotkey='5Hjnirznzz3rYKodKCR2F2qzXC8ZjwZbUrt8w7CdbcPgYXMV', model='unsloth/gemma-3-4b-it', block=5807815, chute={'chute_id': '4d636d79-623c-5688-b57c-264f9e53d85d', 'name': 'unsloth/gemma-3-4b-it', 'public': True, 'version': '8ff77f39-79f4-5aac-b2bd-03e37b7a3556', 'tool_description': None, 'slug': 'chutes-unsloth-gemma-3-4b-it', 'standard_template': 'vllm', 'cord_ref_id': None, 'created_at': '2025-03-12T20:06:51.488258Z', 'updated_at': '2025-05-16T13:15:12.915543Z', 'image': {'image_id': '1075450f-1c30-522e-a58a-3a7c775a008c', 'name': 'sglang', 'tag': '0.4.6.post4', 'public': True, 'status': 'built and pushed', 'created_at': '2025-05-14T16:04:26.948160Z', 'build_started_at': '2025-05-14T16:04:28.960327Z', 'build_completed_at': '2025-05-14T16:24:14.068098Z', 'user': {'username': 'chutes', 'user_id': 'dff3e6bb-3a6b-5a2b-9c48-da3abcd5ca5f', 'logo_id': '24ef9084-7739-400c-aab6-135463a8c2d

In [14]:
import os
import sys
import json
import time
import click
import random
import aiohttp
import asyncio
import bittensor as bt
from dotenv import load_dotenv
from collections import defaultdict
from abc import ABC, abstractmethod
from alive_progress import alive_bar
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional, Union


def get_average_scores(results):
    stats = defaultdict(list)
    for r in results:
        stats[r.miner.uid].append(r.evaluation.score)
    return {uid: sum(scores)/len(scores) if scores else 0 for uid, scores in stats.items()}

In [15]:
get_average_scores(results)

{5: 0.0, 6: 0.0}

In [ ]:
scores: Dict[str, float] = {}
    async def step():
        all_miners = await miners(no_null=True)
        if len(all_miners) < 2:
            raise RuntimeError("Not enough miners to select 2 unique miners.")
        selected_uids = random.sample(list(all_miners.keys()), 2)
        miner1 = all_miners[selected_uids[0]]
        miner2 = all_miners[selected_uids[1]]
        results = await run(
            challenges = SAT().many(1),
            miners = [miner1,miner2]
        )
        scores = []

{5: Miner(uid=5, hotkey='5Hjnirznzz3rYKodKCR2F2qzXC8ZjwZbUrt8w7CdbcPgYXMV', model='unsloth/gemma-3-4b-it', block=5807815, chute=None)}

In [9]:
await get_chute(  'unsloth/gemma-3-4b-it')

{'chute_id': '4d636d79-623c-5688-b57c-264f9e53d85d',
 'name': 'unsloth/gemma-3-4b-it',
 'public': True,
 'version': '8ff77f39-79f4-5aac-b2bd-03e37b7a3556',
 'tool_description': None,
 'slug': 'chutes-unsloth-gemma-3-4b-it',
 'standard_template': 'vllm',
 'cord_ref_id': None,
 'created_at': '2025-03-12T20:06:51.488258Z',
 'updated_at': '2025-05-16T13:15:12.915543Z',
 'image': {'image_id': '1075450f-1c30-522e-a58a-3a7c775a008c',
  'name': 'sglang',
  'tag': '0.4.6.post4',
  'public': True,
  'status': 'built and pushed',
  'created_at': '2025-05-14T16:04:26.948160Z',
  'build_started_at': '2025-05-14T16:04:28.960327Z',
  'build_completed_at': '2025-05-14T16:24:14.068098Z',
  'user': {'username': 'chutes',
   'user_id': 'dff3e6bb-3a6b-5a2b-9c48-da3abcd5ca5f',
   'logo_id': '24ef9084-7739-400c-aab6-135463a8c2d9',
   'created_at': '2024-12-03T11:31:39.181725Z',
   'logo': 'https://logos.chutes.ai/logos/24ef9084-7739-400c-aab6-135463a8c2d9.webp'},
  'logo_id': None,
  'logo': None},
 'user':